# 3. Optimal Top N

INCOMPLETE Example

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../..')

In [4]:
from pathlib import Path

In [5]:
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import OptimalTopNController
from vimms.Environment import Environment
from vimms.Common import *
from vimms.SequenceManager import *

### Load the data

In [6]:
data_dir = os.path.abspath(os.path.join(os.getcwd(),'..','..','tests','integration','fixtures'))
dataset_file = os.path.join(data_dir, 'QCB_22May19_1.p')
dataset = load_obj(dataset_file)
ps = load_obj(Path(data_dir,'peak_sampler_mz_rt_int_beerqcb_fragmentation.p'))
box_file = str(Path(data_dir, 'QCB_22May19_1_pp.csv'))

### Parameters

In [7]:
rt_range = [(0, 1440)]
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]

isolation_window = 1
N = 10
rt_tols = 15
mz_tols = 10
min_ms1_intensity = 1E3

### Run Optimal Top N Controller

In [8]:
mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, ps)
controller = OptimalTopNController(POSITIVE, N, isolation_window, mz_tols, rt_tols, min_ms1_intensity, box_file)

2020-08-12 18:47:39.813 | DEBUG    | vimms.Controller.topN:_load_boxes:444 - Loaded 3523 boxes


In [9]:
len(controller.boxes)

3523

In [10]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

(1440.000s) ms_level=1 N=10 DEW=15: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1439.600000000068/1440 [00:39<00:00, 36.85it/s]


In [11]:
set_log_level_debug()
mzml_filename = 'optimaltopn_controller1.mzML'
out_dir = os.path.join(os.getcwd(), 'results')
env.write_mzML(out_dir, mzml_filename)

2020-08-12 18:48:21.853 | DEBUG    | vimms.Environment:write_mzML:162 - Writing mzML file to C:\Users\Vinny\work\vimms\demo\03. MultiSampleMethods\results\optimaltopn_controller1.mzML
2020-08-12 18:51:17.457 | DEBUG    | vimms.Environment:write_mzML:169 - mzML file successfully written!


In [58]:
len(controller.boxes)
new_boxes = controller.boxes

In [59]:
mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, ps)

controller = OptimalTopNController(POSITIVE, N, isolation_window, mz_tols, rt_tols, min_ms1_intensity, new_boxes)

In [60]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

(1440.200s) ms_level=1 N=3 DEW=15: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1439.8000000000536/1440 [00:36<00:00, 39.04it/s]


In [61]:
len(controller.boxes)

2718

In [ ]:
set_log_level_debug()
mzml_filename = 'optimaltopn_controller2.mzML'
out_dir = os.path.join(os.getcwd(), 'results')
env.write_mzML(out_dir, mzml_filename)

### Run Top N Controller

In [15]:
output_dir = 'C:/Users/Vinny/work/mzmine_files/NaiveMultiSampleExperiment'
mzmine_command = 'C:\\Users\\Vinny\\work\\MZmine-2.40.1\\MZmine-2.40.1\\startMZmine_Windows.bat'

In [18]:
controller_method = 'TopNController'
controller_params = {"ionisation_mode": POSITIVE,
                       "N": 10,
                       "mz_tol": 10,
                       "rt_tol":30,
                       "min_ms1_intensity": 1.75E5,
                       "isolation_width": 1}
mass_spec_params = {'ionisation_mode': POSITIVE,
                    'peak_sampler': ps,
                    'add_noise': False,
                    'isolation_transition_window': 'rectangular',
                    'isolation_transition_window_params': None}
dataset_files = [dataset_file, dataset_file]

In [21]:
vsm = VimmsSequenceManager(None, None, output_dir, progress_bar=True, ms1_picked_peaks_file=None,mzmine_command=mzmine_command)
nsm = NaiveMultiSampleExperiment(vsm, controller_method, mass_spec_params, dataset_files, controller_params, aligner_rt_tolerance=0)

(0.400s) ms_level=1 N=10 DEW=30:   0%|                                                                                                                                                                   | 0.4/1440 [00:00<00:03, 414.15it/s]


AttributeError: 'bool' object has no attribute 'get_mz'

In [ ]:
nsm.cumulative_score